In [1]:
import openai
import re
import httpx
import os
import os
os.environ["OPENAI_API_KEY"] = " "
assert "OPENAI_API_KEY" in os.environ
from openai import OpenAI

client = OpenAI()

# Agent from Scratch

In [3]:
class Agent:
    def __init__(self, input_prompt=""):
        self.input_prompt = input_prompt ## prompt to LLM to explain task to perform
        self.messages = [] ## cache of messages to keep track
        if self.input_prompt:
            self.messages.append({"role": "system", "content": input_prompt})

    def __call__(self, message): ## user message/question
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content

In [12]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [13]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

## One Agent Iteration

In [17]:
abot = Agent(prompt)

### manually running one iteration
result = abot("How much does a toy poodle weigh?") ## ask question
print(result) ## output suggests to take an action average_dog_weight(Toy Poodle)

result = average_dog_weight("Toy Poodle") ## take the action
next_prompt = "Observation: {}".format(result) ## make an Observation -> with result of action
abot(next_prompt) ## call llm with the observation
abot.messages

Thought: I should look up the average weight of a toy poodle using the average_dog_weight function.
Action: average_dog_weight: Toy Poodle
PAUSE


[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [20]:
abot = Agent(prompt)

### manually running one iteration
def agent_iteration(abot, question):
    result = abot(question) ## ask question
    print(result) ## Get Thought and potential Action llm thinks we should run
    
    ## find actions from output string using regex
    action_re = re.compile('^Action: (\w+): (.*)$')
    actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
    print(actions)
    action, action_input = actions[0].groups()
    print(action, action_input)
    
    ##execute action
    observation = known_actions[action](action_input)
    print("Observation:", observation)
    next_prompt = "Observation: {}".format(observation)
    result = abot(next_prompt) ## call llm with the observation
    print(result)

In [22]:
question = "How much does a toy poodle weigh?"
agent_iteration(abot, question)

Thought: I need to look up the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE
[<re.Match object; span=(0, 38), match='Action: average_dog_weight: Toy Poodle'>]
average_dog_weight Toy Poodle
Observation: a toy poodles average weight is 7 lbs
Answer: A Toy Poodle weighs an average of 7 lbs.


## Loop for continuous Agents to solve tasks

In [25]:
### copy agent iteration fn built above
def agent_iteration(abot, question):
    result = abot(question) ## ask question
    print(result)
    action_re = re.compile('^Action: (\w+): (.*)$')
    actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)]
    if actions:
        action, action_input = actions[0].groups()
        observation = known_actions[action](action_input)
        print("Observation:", observation)
        next_prompt = "Observation: {}".format(observation)
        return next_prompt
    else:
        return None

In [ ]:
### Continuous agent

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        next_prompt = agent_iteration(abot, next_prompt)
        if next_prompt is None: return

In [27]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I already have the average weight of a Border Collie, which is 37 lbs. Now I need to find the average weight of a Scottish Terrier to calculate their combined weight.
Action: average_dog_weight: Scottish Terrier
PAUSE
Observation: Scottish Terriers average 20 lbs
Thought: I now have the average weights of both a Border Collie (37 lbs) and a Scottish Terrier (20 lbs). I need to add these weights together to find their combined weight.
Action: calculate: 37 + 20
PAUSE
Observation: 57
Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.


In [5]:
## LangGraph
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults


tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

········
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [ ]:
## LangGraph imports
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults


tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class Agent:

    def __init__(self, model, tools, input_prompt=""):
        self.system = input_prompt
        
        #### BUILD GRAPH #######
        graph = StateGraph(AgentState) ## initialize state of agent
        graph.add_node("llm", self.call_openai) ## add llm node which calls function 'call_openai'
        graph.add_node("action", self.take_action) ## add action node which calls function 'take_action'
        graph.add_conditional_edges("llm", ## add conditional edge, first argument is node where edge starts i.e "llm"
        self.exists_action, ## function to call to understand what to do next
            {True: "action", False: END} ## if function returns True, take the action other wise end
        )